In [1]:
import pandas as pd
from pathlib import Path

In [8]:
BASE_DIR = Path().resolve().parent
raw_dir = (BASE_DIR / "./app/data/raw").resolve()
processed_dir = (BASE_DIR / "./app/data/processed").resolve()

tiktok_path = raw_dir / "tiktok_data.csv"
youtube_path = raw_dir / "youtube_data.csv"
reddit_path = raw_dir / "reddit_data.csv"

processed_dir.mkdir(parents=True, exist_ok=True)

In [11]:
df_tiktok = pd.read_csv(tiktok_path)
df_youtube = pd.read_csv(youtube_path)
df_reddit = pd.read_csv(reddit_path)

# TikTok: nach TikTok-Video-ID
df_tiktok.drop_duplicates(subset="id", inplace=True)

# YouTube: nach Video-ID
df_youtube.drop_duplicates(subset="video_id", inplace=True)

# Reddit: nach Reddit-Post-URL
df_reddit.drop_duplicates(subset="url", inplace=True)

print(f"Shapes - TikTok: {df_tiktok.shape}, YouTube: {df_youtube.shape}, Reddit: {df_reddit.shape}")

display(df_tiktok)
display(df_youtube)
display(df_reddit)


Shapes - TikTok: (94, 10), YouTube: (61, 10), Reddit: (171, 8)


,id,description,author_username,author_id,likes,shares,comments,plays,video_url,created_time
0,7493469801654881542,#vairalvideo_foryoupage #🇦🇫ازبک_تاجک_پشتون_تر...,afgcap.cut,7461541069958153234,15800,451,258,365200,https://v16-webapp-prime.tiktok.com/video/tos/...,1744709406
1,7489427780397010198,#imapoliceofficer #tensheet #foryou #viral #fy...,backwheelbandit69,7416366442453632032,4300000,303800,11300,41900000,https://v16-webapp-prime.tiktok.com/video/tos/...,1743768297
2,7492000423641959685,#CapCut #قوالب_كاب_كات_جاهزه_للتصميم__🌴♥ #كاب_...,noordeen_cap_cat_0_1,7322835376556442629,58400,1576,451,1300000,https://v16-webapp-prime.tiktok.com/video/tos/...,1744367290
3,7472584144510373125,i think it was a bad idea,maligoshik,7014608336423617542,18000000,1300000,24000,168600000,https://v16-webapp-prime.tiktok.com/video/tos/...,1739846584
4,7461927005689302280,welcome to the thanos world!! #squidgame #squi...,team_thanos_player230,7455509281098515474,261400,9526,13000,12400000,https://webapp-sg.tiktok.com/bf63b8aa40b9ff0ca...,1737365272
...,...,...,...,...,...,...,...,...,...,...
185,7492691034397314311,#CapCut #انا_شوري_عل_زلم_يصبح_قرار,w_16.n,7418886743216210952,141800,4684,702,3300000,https://v16-webapp-prime.tiktok.com/video/tos/...,1744528081
194,7467391747606285600,For You Trick #viralvideoontiktok #foryupagе ...,techskills9,7461844247844848673,18200,2404,1376,965200,https://webapp-i18n.tiktok.com/ee5c3ed400b5354...,1738637688
199,7482412775692635397,FIFA STREET?🇧🇷 estilo brasileiro,arthurzinnv,7113702377895592965,3800000,269800,18600,50000000,https://v16-webapp-prime.tiktok.com/video/tos/...,1742134987
203,7475293603305196822,😤😤😤😤😤😤,adamferenc69,7321824335185658912,51300,20000,2106,1500000,NaN,1740477427


,video_id,title,description,channel_title,published_at,view_count,like_count,comment_count,url,scraped_at
0,4sHs9ujo1eg,Die Heuchelei der Stars & Influencer auf dem C...,Das Coachella Festival ist grade bei vielen In...,Alicia Joe,2025-04-21T16:02:49Z,322656,19853,1352,https://www.youtube.com/watch?v=4sHs9ujo1eg,2025-04-22 22:06:12.302112
1,gTA6gQIC39A,Warum ausgerechnet Katy Perry jetzt ins All fl...,Anzeige | Ladet euch für eure nächste Reise Ai...,Desy,2025-04-21T17:30:01Z,172254,10729,1183,https://www.youtube.com/watch?v=gTA6gQIC39A,2025-04-22 22:06:12.302112
2,ncXAUBGV8JI,"Problemmotoren von VW, Ford, Stellantis und Co...","Ein nasser, in Öl geführter Zahnriemen schien ...",auto motor und sport,2025-04-21T15:01:16Z,159413,2657,768,https://www.youtube.com/watch?v=ncXAUBGV8JI,2025-04-22 22:06:12.302112
3,dFKCoqPjJ28,Fynn Kliemanns Comeback ist ein Fiebertraum,Fynn Kliemann ist zurück. Der DIY-Künstler und...,Der Dunkle Parabelritter,2025-04-21T17:30:02Z,169827,10718,1113,https://www.youtube.com/watch?v=dFKCoqPjJ28,2025-04-22 22:06:12.302112
4,gzCXZID3AlY,Streit mit PAULA ? Kontakt mit leiblichen ELTE...,"Anzeige | \nHey, wie versprochen kommen hier a...",Beauty Benzz,2025-04-21T14:00:58Z,70563,2880,293,https://www.youtube.com/watch?v=gzCXZID3AlY,2025-04-22 22:06:12.302112
...,...,...,...,...,...,...,...,...,...,...
56,CF_1py1maEQ,Highlights! | NUNES FIRES INJURY TIME WINNER A...,Matheus Nunes scored a dramatic 94th-minute wi...,Man City,2025-04-22T23:00:11Z,1039070,25293,800,https://www.youtube.com/watch?v=CF_1py1maEQ,2025-04-23 12:59:12.884569
57,rOxHxDDTKcs,FC Barcelona - Mallorca | LaLiga | DAZN Highli...,Der Tabellenführer der Primera Division empfän...,DAZN Fußball,2025-04-22T21:43:05Z,181433,6161,325,https://www.youtube.com/watch?v=rOxHxDDTKcs,2025-04-23 12:59:12.884569
58,REYF3QFVHVc,mgk - Come Pick Me Up (Cover),mgk - Come Pick Me Up (Cover)\n\nDirected by: ...,mgk,2025-04-22T18:05:16Z,261958,24565,2603,https://www.youtube.com/watch?v=REYF3QFVHVc,2025-04-23 12:59:12.884569
59,k4utXKvOpYM,Watch the Destiny 2: The Edge of Fate Reveal o...,"The unknown beckons. \n\nJoin us on May 6, 202...",Destiny 2,2025-04-22T15:01:12Z,214078,7940,970,https://www.youtube.com/watch?v=k4utXKvOpYM,2025-04-23 12:59:12.884569


,subreddit,title,text,score,comments,created,url,scraped_at
0,all,[Charania] BREAKING: The Phoenix Suns have fir...,BREAKING: The Phoenix Suns have fired coach Mi...,7056,1010,2025-04-14 18:03:04,https://www.reddit.com/r/nba/comments/1jz2ize/...,2025-04-14 20:31:29.692066
1,all,I did the math and what I found was shocking. ...,NaN,6832,206,2025-04-14 16:20:10,https://www.reddit.com/r/texas/comments/1jz015...,2025-04-14 20:31:29.692066
2,all,AITA for breaking up with my bf after he didn'...,I (19F) was with my boyfriend (26M) and we wer...,3965,1485,2025-04-14 17:19:43,https://www.reddit.com/r/AITAH/comments/1jz1gp...,2025-04-14 20:31:29.692066
3,all,Why does nobody care that the Trump Admin is d...,Regarding the facilitating of the return of Ab...,3724,1275,2025-04-14 15:43:40,https://www.reddit.com/r/AskUS/comments/1jyz7c...,2025-04-14 20:31:29.692066
4,all,Some upper management walked in on a random go...,"I don’t even know how to write this correctly,...",4046,235,2025-04-14 14:46:57,https://www.reddit.com/r/antiwork/comments/1jy...,2025-04-14 20:31:29.692066
...,...,...,...,...,...,...,...,...
173,popular,I’m a nurse and I 100 percent judge people bas...,"Okay, so I know this might sound weird, but I ...",2123,813,2025-04-23 05:23:51,https://www.reddit.com/r/confession/comments/1...,2025-04-23 12:56:52.751422
175,trendingreddits,https://shrinkme.ink/0VUpb,NaN,1,0,2025-04-22 13:04:40,https://www.reddit.com/r/TrendingReddits/comme...,2025-04-23 12:56:52.751422
176,trendingreddits,https://shrinkme.ink/QXl9OfM,😂,1,0,2025-04-22 08:55:37,https://www.reddit.com/r/TrendingReddits/comme...,2025-04-23 12:56:52.751422
177,trendingreddits,flex,https://preview.redd.it/tygul9qf97we1.png?widt...,1,0,2025-04-21 16:39:22,https://www.reddit.com/r/TrendingReddits/comme...,2025-04-23 12:56:52.751422
